In [3]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.callbacks import StreamingStdOutCallbackHandler

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

chat = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=api_key,
    temperature=0.8,
    )

In [58]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

generate_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an creative and artistic poet. Write a poem about the following programming languages."),
    ("user", "language: {lang}"),
])

explain_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a literary critic who can clearly and insightfully describe a given poem. Show the given poem to the user and add a description of it."),
    ("user", "poem: {poem}"),
])

generate_chain = generate_prompt | chat
explain_chain = explain_prompt | chat
final_chain = {"poem": generate_chain} | explain_chain

final_chain.invoke({"lang": "Java"})

AIMessage(content='Here is the poem you provided:\n\n**O Java, stalwart, strong, and vast,**\n**A titan built to surely last.**\n**With curly braces, neatly bound,**\n**Your object-oriented world profound.**\n\n**From humble start, a platform\'s might,**\n**You power systems, day and night.**\n**The Android realm, your vibrant hue,**\n**A billion screens reflecting you.**\n\n**Though verbose some might claim you be,**\n**Your structure grants stability.**\n**A fortress built of class and type,**\n**Where errors seldom leap and swipe.**\n\n**From enterprise to simple game,**\n**Your versatility proclaims**\n**A legacy of code so deep,**\n**While newer tongues their secrets keep.**\n\n**Yet in your strength, a certain grace,**\n**A steady hand, a measured pace.**\n**O Java, ever strong and true,**\n**Your reign continues, ever new.**\n\n\nThis poem is a celebratory ode to the Java programming language.  It employs a consistently elevated tone, using elevated diction ("stalwart," "titan,"

In [20]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate

examples = [
    {
        "film": "Avatar",
        "answer": """
        I know this:
        Directed by: James Cameron
        Starring: Sam Worthington, Zoe Saldana, Stephen Lang, Michelle Rodriguez, Sigourney Weaver
        Budget: $237 million
        Box office: $2,923 billion
        Genre: Science fiction
        """
    },
    {
        "film": "The Avengers",
        "answer": """
        I know this:
        Directed by: Joss Whedon
        Starring: Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth, Scarlett Johansson
        Budget: $220-225 million
        Box office: $1,521 billion
        Genre: Superhero
        """
    },
    {
        "film": "Your Name",
        "answer": """
        I know this:
        Directed by: Makoto Shinkai
        Starring: Ryunosuke Kamiki, Mone Kamishiraishi
        Budget: ¥750 million (≈$7.5 million)
        Box office: $405.3 million
        Genre: Anime, Fantasy
        """
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("user", "What do you know about {film}?"),
    ("ai", "{answer}"),
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a film expert. You give short answers and add a short plot summary of given film."),
    example_prompt,
    ("user", "What do you know about {film}?")
])

chain = final_prompt | chat

chain.invoke({
    "film": "Frozen"
})

AIMessage(content='I know this:\n\nDirected by: Chris Buck, Jennifer Lee\nStarring: Kristen Bell, Idina Menzel, Jonathan Groff\nBudget: $150 million\nBox office: $1.28 billion\nGenre: Animated musical fantasy\n\nPlot Summary: Two sisters, Elsa and Anna, who are princesses, have magical powers. Elsa can create ice and snow, but she accidentally injures Anna.  As a result, Elsa isolates herself, and when her powers are revealed, she accidentally plunges their kingdom into an eternal winter. Anna sets out on a journey to find Elsa and reverse the curse.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--730e7268-d4b1-4508-a1cf-91a152aebeaf-0', usage_metadata={'input_tokens': 247, 'output_tokens': 126, 'total_tokens': 373, 'input_token_details': {'cache_read': 0}})

In [20]:
from langchain_core.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.messages import HumanMessage, AIMessage

examples = [
    {
        "film": "Top Gun",
        "answer": "🛩👨‍✈️🔥"
    },
    {
        "film": "The Godfather",
        "answer": "👨‍👨‍👦🔫🍝"
    },
    {
        "film": "Titanic",
        "answer": "🚢💔🌊"
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("user", "{film}"),
    ("ai", "{answer}"),
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a film expert. Your answer should be up to 3 emojis"),
    example_prompt,
    MessagesPlaceholder(variable_name="history"),
    ("human", "{film}")
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | chat

def invoke_chain(film):
    result = chain.invoke({"film": film})
    memory.save_context(
        {"input": film},
        {"output": result.content}
    )
    print(result.content)

invoke_chain("Interstellar")
invoke_chain("Jurassic World")

🪐👨‍🚀⏳
🦖🦕🌴


In [21]:
invoke_chain("The previous movie I asked")

🦖🦕🌴


In [1]:
from langchain_community.document_loaders.text import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

loader = TextLoader("./files/doc.txt")
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100
)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
cache_dir = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

docs = loader.load_and_split(text_splitter=splitter)
vectorstore = FAISS.from_documents(docs, cached_embeddings)



In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    return_messages=True,
)

retriever = vectorstore.as_retriever()

def load_memory(_):
    return memory.load_memory_variables({})["history"]

def get_retriever(_):
    return vectorstore.as_retriever()

def invoke_chain(question):
    result = chain.invoke({
        "context": vectorstore.similarity_search(question),
    "question": question,
    })
    memory.save_context(
        {"input": question},
        {"output": result.content}
    )
    print(result.content)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

chain = RunnablePassthrough.assign(history=load_memory) | prompt | chat

invoke_chain("Is Aaronson guilty?")
invoke_chain("What message did he write on the table?")
invoke_chain("Who is Julia?")

The text states that Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with, and that Winston had never seen the photograph that disproved their guilt because it had never existed.
He wrote "2 + 2 = 5" in the dust on the table.
Julia is a woman with whom Winston had a forbidden relationship.
